In [81]:
import pandas as pd
import os as os

In [82]:
def check_file_exists() -> str:
    while True:
        filename = input("Enter filename. Relative paths are acceptable ... \n")
        if os.path.isfile(filename):
            print("File Exists. OK to continue")
            return filename
        else:
            print(f"File {filename} does not exist. Check path and filename.")

# Get the file name and read into a df if exists
xls = check_file_exists()
excel_map = pd.read_excel(xls, sheet_name=None)

# Clean the data of trailing spaces for the Categories column 
for sheet, df in excel_map.items():
    if 'Category' in df.columns:
        # Strip trailing spaces from the 'Category' column values
        df['Category'] = df['Category'].str.rstrip()

Enter filename. Relative paths are acceptable ... 
tyb2425.xlsx
File Exists. OK to continue


In [83]:
# Open up excel into an object
# This takes a long time - in production might use different approach
# Was taking a long time due to capacity blank cells pasted into worksheets!
#xls = pd.ExcelFile('tyb2122.xlsx')

# Read sheets into a dictionary of sheets
#excel_map = pd.read_excel(xls, sheet_name=None)
#print(excel_map.keys())

In [84]:
# Copy into a dict just because of naming below. No real need to do this apart from renaming everything below
df_dict = excel_map
print(type(df_dict))

<class 'dict'>


In [85]:
print(df_dict['Aug 2024'])

         Date         Category                         Detail In / Out  \
0  2024-08-01         Mortgage                   Principality        -   
1  2024-08-01      Council Tax                       Gywnedd         -   
2  2024-08-01      Electricity                           Fuse        -   
3  2024-08-01            Water                      Dwr Cymru        -   
4  2024-08-01        Broadband                       Vodafone        -   
5  2024-08-01       TV Licence                             TV        -   
6  2024-08-01          Laundry               Pressing Matters        -   
7  2024-08-05      Furnishings                Outdoor Parasol        -   
8  2024-08-07          Welcome                           Milk        -   
9  2024-08-17          Welcome                           Cake        -   
10 2024-08-03          Welcome                        Flowers        -   
11 2024-08-17          Welcome                        Flowers        -   
12 2024-08-04          Welcome        

In [86]:
# Create a list of sheets to ignore
# Iterate over the sheets in the dict created above (misnamed stuff when starting out is all)
# Exclude the sheet if it's in the list
# Do some DQ analysis
# Otherwise - Group the data in the current sheet. Convert in-line to a dataframe
# Concatenate these as we go along
# Group the final sheet up to category level as it is the complete set

exclude_sheets = ['Template', 'Lists']

month_totals_df= pd.DataFrame()
totals_df= pd.DataFrame()

for k in df_dict.keys():
    #print(f"{k =}")
    if k in exclude_sheets:
        #print("Excluded")
        continue
    else:
        
        # Check actuals column for nulls:
        check_nan = df_dict[k]['Actual'].isnull().values.any()
        if check_nan:
            print(f'Null Actuals in sheet: {k}')
        try:
            current_month_df = pd.DataFrame(df_dict[k].groupby('Category')['Actual'].sum())
        except:
            print(f'Error grouping: {k}')
        #Add current month to new dataframe
        current_month_df['month'] = k
        #display(current_month_df.head(10))
        #display(totals_df)
        month_totals_df = pd.concat([month_totals_df, current_month_df], axis=0) #, ignore_index=False)

# Group the months
totals_df = pd.DataFrame(month_totals_df.groupby('Category')['Actual'].sum()).reset_index()
#print(totals_df)
rental_categories = ['Airbnb', 'Cottages', 'Private']



In [87]:
# Format floats to 2 places:
month_totals_df.style.format({"Actual" : ".2f"})
totals_df.style.format({"Actual" : ".2f"})
# Output to csv
month_totals_df.to_csv('categories_all_months.csv', float_format='%.2f')

totals_df.to_csv('category_totals.csv', float_format='%.2f')

In [88]:
# Format floats to 2 places:
# month_totals_df.style.format({"Actual" : "{.2f}"})
month_totals_df.style.format({"Actual": "{:.2f}".format})
#totals_df.style.format({"Actual": "{.2f}"})
totals_df.style.format({"Actual": "{:.2f}"})


,Category,Actual
0,Advertising,-12.79
1,Airbnb,12690.08
2,Broadband,-308.22
3,Council Tax,0.00
4,Electricity,-1884.97
5,Furnishings,-309.89
6,Household,-410.94
7,Insurance,-369.57
8,Laundry,-1188.70
9,Logs,-190.00


In [89]:
print(totals_df)

           Category    Actual
0       Advertising    -12.79
1            Airbnb  12690.08
2         Broadband   -308.22
3       Council Tax      0.00
4       Electricity  -1884.97
5       Furnishings   -309.89
6         Household   -410.94
7         Insurance   -369.57
8           Laundry  -1188.70
9              Logs   -190.00
10      Maintenance  -1941.30
11         Mortgage  -4770.23
12          Private   4116.00
13       TV Licence   -178.25
14  Travel Expenses   -442.26
15            Water   -356.50
16          Welcome   -482.17


In [90]:
print(f'month_totals: {month_totals_df.dtypes}')
print(f'totals: {totals_df.dtypes}')


month_totals: Actual    float64
month      object
dtype: object
totals: Category     object
Actual      float64
dtype: object


In [91]:
print(totals_df.head(100))


           Category    Actual
0       Advertising    -12.79
1            Airbnb  12690.08
2         Broadband   -308.22
3       Council Tax      0.00
4       Electricity  -1884.97
5       Furnishings   -309.89
6         Household   -410.94
7         Insurance   -369.57
8           Laundry  -1188.70
9              Logs   -190.00
10      Maintenance  -1941.30
11         Mortgage  -4770.23
12          Private   4116.00
13       TV Licence   -178.25
14  Travel Expenses   -442.26
15            Water   -356.50
16          Welcome   -482.17


april_df = df['April 21']
print(april_df.head(5))

april_df.columns = ['Date', 'Category', 'Detail', 'In / Out', 'Amount', 'Actual', 'Notes',
       'Unnamed: 7', 'Unnamed: 8']
print(april_df.head(5))

In [92]:
examine_df = df_dict['Mar 2025']
examine_df

,Date,Category,Detail,In / Out,Amount,Actual,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,2025-03-01,Mortgage,Principality,-,370.18,-370.18,NaN,NaN,NaN
1,2025-03-01,Council Tax,Gywnedd,-,0.00,0.00,NaN,In,946.00
2,2025-03-01,Electricity,E-on,-,217.03,-217.03,NaN,Out,-1770.89
3,2025-03-01,Water,Dwr Cymru,-,30.50,-30.50,NaN,NaN,NaN
4,2025-03-01,Broadband,Vodafone,-,25.00,-25.00,NaN,Total,-824.89
5,2025-03-01,TV Licence,TV,-,15.00,-15.00,NaN,NaN,NaN
6,2025-03-01,Laundry,Pressing Matters,-,0.00,0.00,NaN,NaN,NaN
7,2025-03-15,Furnishings,Replacement Vase,-,10.50,-10.50,NaN,NaN,NaN
8,2025-03-24,Maintenance,Chimney Sweep,-,130.00,-130.00,NaN,NaN,NaN
9,2025-03-24,Maintenance,Glass Repair Woodburner,-,70.00,-70.00,NaN,NaN,NaN


In [93]:
from IPython.display import clear_output

try:
    del df_dict["Template"]
    del df_dict["Lists"]
except Exception as e:
    print (e)
    
    
for month, month_df in df_dict.items():
    clear_output(wait=True)
    print(f"Displaying - {month}")
    #month_df = df_dict['January']
    display(month_df)
    #per_month_actuals = month_df.groupby('Category')['Actual'].sum()
    #per_month_actuals.head(100)
    input("Press ENTER to see next month")
    


Displaying - Apr 2024


,Date,Category,Detail,In / Out,Amount,Actual,Notes,Unnamed: 7,Unnamed: 8
0,2024-04-01,Mortgage,Principality,-,409.53,-409.53,NaN,NaN,NaN
1,2024-04-01,Council Tax,Gywnedd,-,0.00,0.00,NaN,NaN,NaN
2,2024-04-01,Electricity,E-On Next,-,225.89,-225.89,NaN,NaN,NaN
3,2024-04-01,Water,Dwr Cymru,-,21.00,-21.00,NaN,In,1662.90
4,2024-04-01,Broadband,Vodafone,-,27.16,-27.16,NaN,Out,-1396.71
5,2024-04-01,TV Licence,TV,-,13.25,-13.25,NaN,NaN,NaN
6,2024-04-01,Laundry,Pressing Matters,-,89.64,-89.64,NaN,Total,266.19
7,2024-04-07,Welcome,Kindling,-,36.00,-36.00,NaN,NaN,NaN
8,2024-04-02,Maintenance,Loft Insulation,-,138.00,-138.00,NaN,NaN,NaN
9,2024-04-01,Welcome,Milk,-,3.00,-3.00,NaN,NaN,NaN


Press ENTER to see next month


In [94]:
# TODO Per month


In [95]:
month_totals_df.loc['Household']

,Actual,month
Category,,
Household,-13.73,Mar 2025
Household,-60.26,Jan 2025
Household,-27.74,Dec 2024
Household,-27.86,Nov 2024
Household,-9.98,Oct 2024
Household,-3.99,Sept 2024
Household,-36.02,Aug 2024
Household,-53.63,Jul 2024
Household,-28.86,Jun 2024


In [96]:
# WHERE CLAUSE ?
# Per Category
#display(totals_df)
where_df = month_totals_df.where(month_totals_df['month']=='Apr 2024')
display(where_df)


,Actual,month
Category,,
Broadband,NaN,NaN
Council Tax,NaN,NaN
Electricity,NaN,NaN
Furnishings,NaN,NaN
Household,NaN,NaN
Laundry,NaN,NaN
Maintenance,NaN,NaN
Mortgage,NaN,NaN
Private,NaN,NaN


In [97]:
# Source - https://stackoverflow.com/a
# Posted by starriet 차주녕
# Retrieved 2026-01-18, License - CC BY-SA 4.0
pd.set_option('display.min_rows', 500)  # <-add this!
pd.set_option('display.max_rows', 500)

pd.options.display.max_rows

month_totals_df.head(500)

,Actual,month
Category,,
Broadband,-25.00,Mar 2025
Council Tax,0.00,Mar 2025
Electricity,-217.03,Mar 2025
Furnishings,-10.50,Mar 2025
Household,-13.73,Mar 2025
Laundry,0.00,Mar 2025
Maintenance,-1052.34,Mar 2025
Mortgage,-370.18,Mar 2025
Private,946.00,Mar 2025
